In [ ]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from __init__ import glob_all, plot_map
from __TargetEmbedding__ import auto_delete_hdf5_save_history, Encoder, Decoder
from RandomTesting.__RandomTesting__ import Random_Testing, calculate_extract_loss
from tqdm import tqdm

import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
import keras
import os


warnings.filterwarnings('ignore')
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
tf.config.optimizer.set_jit(True)

# Load

In [ ]:
home_path = '/data1/home/weisting/CycleGan/Experiment5/'

channel_number = 6; train_bs = 12; total_training_time = 30; _=0; extract_nb = 4

EPA73_test_pm25 = np.load(home_path+'348X204/EPA73_Testing_348X204_PM25.npy', mmap_mode='r')[:3000]
station_coordinate = pd.read_csv('RandomTesting/station_coordinate')
ex5_lst = np.load('RandomTesting/extract5_list.npy')
ex10_lst = np.load('RandomTesting/extract10_list.npy')
ex15_lst = np.load('RandomTesting/extract15_list.npy')
ex20_lst = np.load('RandomTesting/extract20_list.npy')

AIR73_train = np.load('AIR73_Align_Training_del.npy', mmap_mode='r')
AIR_train = np.load(home_path+'Fill_Input_Version/Target_Embedding/Fill_TE_Data/AIR_Align_Kriging_Training_del.npy', mmap_mode='r')

AIR73_test = np.load('AIR73_Align_Testing_del.npy', mmap_mode='r')
AIR_test = np.load(home_path+'Fill_Input_Version/Target_Embedding/Fill_TE_Data/AIR_Align_Kriging_Testing_del.npy', mmap_mode='r')

Sat73_train = np.load(home_path+'348X204/SAT73_interlinear_Training_348X204_PM25.npy', mmap_mode='r')
Sat_train = np.load(home_path+'348X204/SAT_interlinear_Training_348X204_PM25.npy', mmap_mode='r')

Sat73_test = np.load(home_path+'348X204/SAT73_interlinear_Testing_348X204_PM25.npy', mmap_mode='r')
Sat_test = np.load(home_path+'348X204/SAT_interlinear_Testing_348X204_PM25.npy', mmap_mode='r')

trainX = np.concatenate([AIR73_train, Sat73_train])
trainY = np.concatenate([AIR_train, Sat_train])

testX = np.concatenate([AIR73_test, Sat73_test])
testY = np.concatenate([AIR_test, Sat_test])

AIRSATALL_Encoder_model = Encoder(channel_number)
AIRSATALL_Decoder_model = Decoder(channel_number)
x = Input(shape=(348, 204, 1))
AIRSATALL_Autoencoder = Model(x,AIRSATALL_Decoder_model(AIRSATALL_Encoder_model(x)))
airsat2code_weight = glob_all(home_path+'Weight/Unpair_Weight_channel'+str(channel_number))[0]
AIRSATALL_Autoencoder.load_weights(airsat2code_weight)
for layer in AIRSATALL_Autoencoder.layers:layer.trainable = False
for layer in AIRSATALL_Encoder_model.layers:layer.trainable = False
for layer in AIRSATALL_Decoder_model.layers:layer.trainable = False

# Main

In [ ]:
for training_time in range(total_training_time):
    K.clear_session()

    path ='Zero_Input_Unpair_Encoder_AIRSATALL_Decoder_channel'+str(channel_number)+'/'+str(training_time)+'/'           

    if not os.path.exists(path):os.makedirs(path)    
    
    AIRSAT73_Encoder_model = Encoder(channel_number)
    AIRSAT73_Autoencoder = Model(x,AIRSATALL_Decoder_model(AIRSAT73_Encoder_model(x)))
    AIRSAT73_Autoencoder.compile(loss='mse',optimizer='adam')    


    filepath=path+'{epoch:04d}-train_{loss:f}-test_{val_loss:f}.h5'
    earlystopper = EarlyStopping(monitor='val_loss', patience=120, verbose=0)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor = 0.2,patience=30,
                                  min_lr = 2e-7,verbose = 0)   

    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, mode='auto')    
    callbacks_list = [earlystopper,reduce_lr,checkpoint]   


    history = AIRSAT73_Autoencoder.fit(trainX,trainY,
                                    validation_data=(testX, testY),
                                    batch_size=train_bs,epochs=200,
                                    callbacks=callbacks_list, verbose=2)      
    
    auto_delete_hdf5_save_history(training_time, _, path, delete=True, plot=False)
    AIRSAT73_Autoencoder.load_weights(glob_all(path)[0])
    
    total_mse, total_mae, total_mape = ([] for _ in range(3))
    for i in tqdm(range(30)):
        matrix_ex ,lats_ex20,lons_ex20=Random_Testing(extract_nb,i,station_coordinate, EPA73_test_pm25, 
                                                      ex5_lst, ex10_lst, ex15_lst, ex20_lst)
        predict_test = AIRSAT73_Autoencoder.predict(matrix_ex)
        mse, mae, mape = calculate_extract_loss(predict_test, EPA73_test_pm25, lats_ex20, lons_ex20)
        total_mse.append(mse); total_mae.append(mae); total_mape.append(mape)
    del predict_test; del matrix_ex
    print('ex20-- mean mse:%.4d_mean mae:%.4d_mean mape:%.4d'
          %(np.mean(total_mse),np.mean(total_mae),np.mean(total_mape)))

    f1 = open(path+'Extractresult'+str(extract_nb)+'.txt', 'a')
    print('----------'+'avg_mse_loss'+'----------',file=f1)
    print('max:  '+str(np.array(total_mse).max()),file=f1)
    print('min:  '+str(np.array(total_mse).min()),file=f1)
    print('mean: '+str(np.array(total_mse).mean()),file=f1)
    print('var:  '+str(np.array(total_mse).var()),file=f1)
    print('std:  '+str(np.array(total_mse).std()),file=f1)
    print('---------------------------',file=f1)

    print('----------'+'avg_mae_loss'+'----------',file=f1)
    print('max:  '+str(np.array(total_mae).max()),file=f1)
    print('min:  '+str(np.array(total_mae).min()),file=f1)
    print('mean: '+str(np.array(total_mae).mean()),file=f1)
    print('var:  '+str(np.array(total_mae).var()),file=f1)
    print('std:  '+str(np.array(total_mae).std()),file=f1)
    print('---------------------------',file=f1)

    print('----------'+'avg_mape_loss'+'----------',file=f1)
    print('max:  '+str(np.array(total_mape).max()),file=f1)
    print('min:  '+str(np.array(total_mape).min()),file=f1)
    print('mean: '+str(np.array(total_mape).mean()),file=f1)
    print('var:  '+str(np.array(total_mape).var()),file=f1)
    print('std:  '+str(np.array(total_mape).std()),file=f1)
    print('---------------------------',file=f1)         
    f1.close()